# データ読み込み

In [ ]:
#

# 前処理

## モデル定義

In [ ]:
import tensorflow as tf
import numpy as np

%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
s_scaler = StandardScaler()

In [ ]:
print(tf.__version__)

In [ ]:
# MLP_for_Basic
basic_size = 7

Basic_MLP_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, input_shape=(basic_size,)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation('relu')
])

Basic_MLP_model.summary()

In [ ]:
# MLP_for_multi_keyword
vec_size = 200
#max_length = 10

inputs = tf.keras.layers.Input(shape=(None, vec_size))
x1 = tf.keras.layers.Masking(mask_value=0.0)(inputs)
x1 = tf.keras.layers.Dense(128, activation='relu')(x1)
x1 = tf.keras.layers.Dense(64, activation='relu')(x1)
x1 = tf.keras.backend.sum(x1, axis=1, keepdims=False)

multi_keyword_model = tf.keras.Model(inputs=inputs, outputs=x1)
multi_keyword_model.summary()
multi_keyword_model.output


In [ ]:
# concat_MLP
merged = tf.keras.layers.concatenate([Basic_MLP_model.output, multi_keyword_model.output])
x2 = tf.keras.layers.BatchNormalization()(merged)
x2 = tf.keras.layers.Dense(64, activation='relu')(x2)
x2 = tf.keras.layers.Dense(64, activation='relu')(x2)
x2 = tf.keras.layers.Dropout(0.5)(x2)
x2 = tf.keras.layers.Dense(1, activation='relu')(x2)#予測値が正なのでreluを通す
concat_MLP_model = tf.keras.Model([Basic_MLP_model.input, multi_keyword_model.input], x2)

concat_MLP_model.summary()

## 関数

In [ ]:
import os
import random
def set_seed(seed):
  os.environ['TF_DETERMINISTIC_OPS'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

In [ ]:
def history_mse_plot(history_with_1000epoch, stop_count, mse_of_train_mean, mse_of_train_10_mean, mse_of_Basic):
  plt.figure(facecolor='w', figsize=(6, 4))
  plt.plot(history_with_1000epoch.history["mse"], )
  plt.plot(history_with_1000epoch.history['val_mse'])
  plt.axhline(mse_of_train_mean, xmin=0.05, xmax=0.95, color='green', lw=0.8)
  plt.axhline(mse_of_train_10_mean, xmin=0.05, xmax=0.95, color='red', lw=0.8)
  plt.axhline(mse_of_Basic, xmin=0.05, xmax=0.95, color='blue', lw=0.8)
  plt.axvline(x=stop_count, ymin=0.05, ymax=0.95, color='orange', lw=0.8)
  plt.title('Model mse')
  plt.ylabel('mse')
  plt.xlabel('Epoch')
  plt.grid()
  plt.legend(['Train', 'Validation', 'Tr mean', 'Tr_10 mean', 'Basic'], loc='upper right')
  plt.show()

In [ ]:
def predict_plot(df, pred_test, pred_train, pred_with_Basic):
  plt.figure(facecolor='w', figsize=(24, 9))
  plt.plot(df["No"], df["favorite_count"])#seriesもOK
  plt.plot(df["No"][len(pred_test):], pred_train, color='orange')
  plt.plot(df["No"][0:len(pred_test)], pred_test, color='red')
  plt.grid()

  plt.figure(facecolor='w', figsize=(16, 6))
  plt.plot(df["No"][0:len(pred_test)], df["favorite_count"][0:len(pred_test)])#seriesもOK
  plt.plot(df["No"][0:len(pred_test)], pred_test, color='red')
  plt.plot(df["No"][0:len(pred_test)], pred_with_Basic, color='orange')
  plt.grid()
  plt.legend(['actual', 'Text', 'Basic'], loc='upper right')
  plt.show()

In [ ]:
def get_baseline(y_test, y_train):
  y_10_mean = np.mean(y_train[0:10])
  y_all_mean = np.mean(y_train)
  y_10_means = np.array([y_10_mean for i in range(y_test.shape[0])])
  y_all_means = np.array([y_all_mean for i in range(y_test.shape[0])])

  y_10_mse = mean_squared_error(y_test, y_10_means)
  y_all_mse = mean_squared_error(y_test, y_all_means)
  return y_all_mean, y_10_mean, y_all_mse, y_10_mse

## compile,fit

In [ ]:
def MLP(X_test, X_train, y_test, y_train, epochs=1000):
  with tf.device('/device:GPU:0'):

    # MLP_for_Basic--------------------
    basic_size = 7

    Basic_MLP_model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, input_shape=(basic_size,)),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Activation('relu')
    ])

    # MLP_for_multi_keyword----------
    vec_size = 200

    inputs = tf.keras.layers.Input(shape=(None, vec_size))
    x1 = tf.keras.layers.Masking(mask_value=0.0)(inputs)
    x1 = tf.keras.layers.Dense(128, activation='relu')(x1)
    x1 = tf.keras.layers.Dense(64, activation='relu')(x1)
    x1 = tf.keras.backend.sum(x1, axis=1, keepdims=False)
    multi_keyword_model = tf.keras.Model(inputs=inputs, outputs=x1)

    # concat_MLP
    merged = tf.keras.layers.concatenate([Basic_MLP_model.output, multi_keyword_model.output])
    x2 = tf.keras.layers.BatchNormalization()(merged)
    x2 = tf.keras.layers.Dense(64, activation='relu')(x2)
    x2 = tf.keras.layers.Dense(64, activation='relu')(x2)
    x2 = tf.keras.layers.Dropout(0.5)(x2)
    x2 = tf.keras.layers.Dense(1, activation='relu')(x2)#予測値は正なのでreluを通す
    concat_MLP_model = tf.keras.Model([Basic_MLP_model.input, multi_keyword_model.input], x2)

    set_seed(42)
    concat_MLP_model.compile(optimizer='adam', 
                  loss='mse', 
                  metrics=[tf.keras.metrics.RootMeanSquaredError(), "mse", "mae"])

    history = concat_MLP_model.fit(X_train, y_train,
                        batch_size=24,
                        epochs=epochs, 
                        verbose=1, 
                        shuffle=True,
                        validation_data=(X_test, y_test))
  return history

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
def MLP_early(X_test, X_train, y_test, y_train, epochs=1000):
  with tf.device('/device:GPU:0'):

    # MLP_for_Basic--------------------
    basic_size = 7

    Basic_MLP_model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, input_shape=(basic_size,)),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Activation('relu')
    ])

    # MLP_for_multi_keyword----------
    vec_size = 200

    inputs = tf.keras.layers.Input(shape=(None, vec_size))
    x1 = tf.keras.layers.Masking(mask_value=0.0)(inputs)
    x1 = tf.keras.layers.Dense(128, activation='relu')(x1)
    x1 = tf.keras.layers.Dense(64, activation='relu')(x1)
    x1 = tf.keras.backend.sum(x1, axis=1, keepdims=False)
    multi_keyword_model = tf.keras.Model(inputs=inputs, outputs=x1)

    # concat_MLP
    merged = tf.keras.layers.concatenate([Basic_MLP_model.output, multi_keyword_model.output])
    x2 = tf.keras.layers.BatchNormalization()(merged)
    x2 = tf.keras.layers.Dense(64, activation='relu')(x2)
    x2 = tf.keras.layers.Dense(64, activation='relu')(x2)
    x2 = tf.keras.layers.Dropout(0.5)(x2)
    x2 = tf.keras.layers.Dense(1, activation='relu')(x2)
    concat_MLP_model = tf.keras.Model([Basic_MLP_model.input, multi_keyword_model.input], x2)

    set_seed(42)
    concat_MLP_model.compile(optimizer='adam', 
                  loss='mse', 
                  metrics=[tf.keras.metrics.RootMeanSquaredError(), "mse", "mae"])
    
    history = concat_MLP_model.fit(X_train, y_train,
                        batch_size=24,
                        epochs=epochs, 
                        verbose=1, 
                        shuffle=True,
                        callbacks=early_stop,
                        validation_data=(X_test, y_test)

  return history, concat_MLP_model.predict(X_train), concat_MLP_model.predict(X_test)

# user1

In [ ]:
user_path = userIDs[0]
user_path

In [ ]:
mse_of_Basic = result_simple_and_multi_keyword[user_path]["basic_mse"]
pred_with_Basic = result_simple_and_multi_keyword[user_path]["basic_predict"]

df_Basic = dict_df_basic[user].drop(columns=["datetime", "year", "month", "day", "hour", "weekday", "unix_time"], axis=1)
df_Text = dict_series_multi_keyword[user]
if df_Basic.shape[0] != df_Text.shape[0]:
  error_list.append(user)
y = df_Basic['favorite_count'].values
X_basic = df_Basic.drop(columns="favorite_count", axis=1)
X_basic = s_scaler.fit_transform(X_basic)#arrayになる
X_text = np.array(df_Text.values.tolist()) #df_Text.valuesだと，array(array())となる

X_basic_test, X_basic_train, X_text_test, X_text_train, y_test, y_train = train_test_split(X_basic, X_text, y, shuffle=False, test_size=0.95)
train_mean, train_10_mean, mse_of_train_mean, mse_of_train_10_mean = get_baseline(y_test, y_train)

In [ ]:
X_train = [X_basic_train, X_text_train]
X_test = [X_basic_test, X_text_test]
history_with_1000epoch = MLP(X_test, X_train, y_test, y_train, epochs=1000)
history, pred_train, pred_test = MLP_early(X_test, X_train, y_test, y_train, epochs=1000)
pred_train = np.ravel(pred_train)
pred_test = np.ravel(pred_test)

In [ ]:
print("訓練データ:", X_basic_train.shape, X_text_train.shape, "テストデータ:", X_basic_test.shape, X_text_test.shape)
history_mse_plot(history_with_1000epoch, len(history.history["loss"]), mse_of_train_mean, mse_of_train_10_mean, mse_of_Basic)
predict_plot(df_Basic, pred_test, pred_train, pred_with_Basic)